In [1]:
# -*- coding: utf-8
import re
import os, sys
import json
from pyhanlp import *
# from stanfordcorenlp import StanfordCoreNLP
import jieba
import time
from lxml import etree
import requests

In [2]:
def ssplit(text):
    text = re.sub('([。！？\?])([^”’])', r"\1\n\2", text)
    text = re.sub('(\.{6})([^”’])', r"\1\n\2", text)
    text = re.sub('(\…{2})([^”’])', r"\1\n\2", text)
    text = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', text)
    return text
start = time.time()
doc_id_counter = 0
csv_list = []
with open("wiki_coding_conv.txt", "r", encoding="utf8") as fr:
    for line in fr:
        line = line.strip().replace("\u3000", "")
        line = re.sub("\s+", " ", line)
        if not line or line == "\n": continue
        if line.startswith("<doc id="):
            doc_id_counter += 1
            doc_dict = {}
            doc_content = []
            doc_info = HanLP.convertToSimplifiedChinese(line)
            doc_info = etree.HTML(doc_info)
            doc_dict["doc_id"] = doc_info.xpath("//@id")[0]
            doc_dict["title"] = doc_info.xpath("//@title")[0]
            continue
        elif line == "</doc>":
            doc_dict["content"]= "\n".join(doc_content)
            csv_list.append(doc_dict)
        else:
            line = HanLP.convertToSimplifiedChinese(line)
            line = ssplit(line)
            for sentence in line.split("\n"):
                if not sentence or sentence == "\n": continue
                sentence = sentence.strip()
                sentence_tok = list(jieba.cut(sentence,use_paddle=True))
                doc_content.append(" ".join(sentence_tok))
end = time.time()
print(f"{end-start}")




Building prefix dict from the default dictionary ...
Loading model from cache /home/fxz/anaconda3/envs/text_preprocess/lib/python3.6/site-packages/jieba/cache/jieba.cache
Loading model cost 0.948 seconds.
Prefix dict has been built successfully.


2611.825758457184


In [5]:
import pandas as pd
result_data = pd.DataFrame(csv_list, columns=["doc_id", "title", "content"])
result_data.to_csv("wiki_tokenized_2.csv",index=True,sep=',')

In [14]:
result_data
# import random
# print(random.choice(result_data["content"]))

,doc_id,title,content
0,13,数学,数学\n数学 是 利用 符号语言 研究 数量 、 结构 、 变化 以及 空间 等 概念 的 ...
1,18,哲学,哲学\n哲学 （ ） 是 研究 普遍 的 、 基础 问题 的 学科 ， 包括 存在 、 知识...
2,21,文学,文学\n文学 （ ） ， 在 最 广泛 的 意义 上 ， 是 任何 单一 的 书面 作品 。...
3,22,历史,历史\n历史 （ 现代汉语 词汇 ， 古典 文言文 称之为 史 ） ， 指 人类 社会 过去...
4,25,计算机科学,计算机科学\n计算机科学 （ ， 有时 缩写 为 ） 是 系统性 研究 信息 与 计算 的 ...
...,...,...,...
1082557,6810537,伊制玛尔,"伊制 玛尔\n伊制 玛尔 （ , Ijm ā ʿ ） ， 或译 为 伊智玛尔 、 公议 、 ..."
1082558,6810542,谢帝,谢帝\n谢帝 ， 原名 蔡镇 鸿 ， 又名 谢 老板 儿 、 Boss X 、 Fat ...
1082559,6810543,史蜀君,史蜀君\n史蜀君 ， 女 ， 北京 人 ， 生于 四川 重庆 。\n中国 电影 导演 ， 编...
1082560,6810553,紫荆山,紫荆山\n紫荆山 可以 指


In [ ]:
# 另一种方式：调用stanfordCoreNLP的分句
docs_dict = {}
error_counter = 0
with StanfordCoreNLP('/home/fxz/JavaLibraries/stanford-corenlp-full-2018-10-05', memory="8g", lang='zh') as nlp:
    start = time.time()
    with open("wiki_extracted.txt", "r", encoding="utf8") as fr:
        for line in fr:
            line = line.strip().replace("\u3000", "")
            if not line or line == "\n": continue
            if line.startswith("<doc id="):
                doc_id = HanLP.convertToSimplifiedChinese(line)
                doc_content = []
                docs_dict[doc_id] = {}
                continue
            elif line == "</doc>":
                for idx, sentence in enumerate(doc_content):
                    docs_dict[doc_id][idx] = sentence
                continue
            else:
                try:
                    line = HanLP.convertToSimplifiedChinese(line)
                    for sentence in nlp.sent_split(line):
                        sentence_tok = list(jieba.cut(sentence,use_paddle=True))
                        doc_content.append(" ".join(sentence_tok))
                except json.JSONDecodeError:
                    error_counter += 1
                    line = HanLP.convertToSimplifiedChinese(line)
                    print(f"{error_counter}: {line}")
                    line = re.sub('([。！？\?])([^”’])', r"\1\n\2", line)
                    line = re.sub('(\.{6})([^”’])', r"\1\n\2", line)
                    line = re.sub('(\…{2})([^”’])', r"\1\n\2", line)
                    line = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', line)
                    print(line)
                    for sentence in line.split("\n"):
                        sentence_tok = list(jieba.cut(sentence,use_paddle=True))
                        doc_content.append(" ".join(sentence_tok))
                    print("retry...success!")
    end = time.time()
    print(f"{end-start}")

with open("wiki_tokenized.json", "w", encoding='utf8') as fw:
    fw.write(json.dumps(docs_dict, ensure_ascii=False, indent=4))